# RAG - ColBERT IR, gpt tubo 3.5

## IR

### 라이브러리 설치 및 임포트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# OpenAI Python 패키지 설치
!pip install openai

# 검색엔진을 위해 ColBERT 사용
!pip install "colbert-ir[faiss-gpu, torch]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 M

In [1]:
!pip install torch==2.4.1 # 런타임 재시작 후 재실행

In [2]:
!pip install torchaudio torchvision --upgrade


In [4]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')


fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 2813, done.
remote: Counting objects: 100% (1314/1314), done.
remote: Compressing objects: 100% (432/432), done.
remote: Total 2813 (delta 1017), reused 972 (delta 878), pack-reused 1499 (from 1)
Receiving objects: 100% (2813/2813), 2.06 MiB | 5.12 MiB/s, done.
Resolving deltas: 100% (1777/1777), done.


In [5]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 55.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found exi

In [6]:
# 필요한 라이브러리 import

from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
import colbert
import json

### ColBER 색인

In [ ]:
# 문서 파일 경로 및 로드
input_file_path = '/content/drive/MyDrive/colbert_ir/documents.jsonl'
collection = []
docid_list = []
src_list = []

# JSONL 파일에서 문서 데이터를 로드하여 리스트로 저장
with open(input_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        document = json.loads(line.strip())
        collection.append(document['content'])  # 'content' 필드만 사용하여 색인
        docid_list.append(document['docid'])
        src_list.append(document['src'])


# 색인 생성 설정
checkpoint = 'colbert-ir/colbertv2.0' # jgkym/ko-colbert-v2-fis
index_name = "test"
doc_maxlen = 500  # 문서의 최대 길이 설정

# 색인 생성
with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks는 사용할 GPU의 수를 지정합니다.
    # ColBERT의 설정을 초기화합니다.
    config = ColBERTConfig(
        doc_maxlen=doc_maxlen,  # 문서의 최대 길이를 지정합니다.
        nbits=2,  # 양자화에 사용되는 비트 수를 설정합니다. 일반적으로 작은 값일수록 인덱스가 더 작아집니다.
        kmeans_niters=4  # k-means 클러스터링의 반복 횟수를 지정합니다. 4는 빠르고 기본적으로 좋은 값입니다. 작은 데이터셋일 경우 더 큰 값을 고려할 수 있습니다.
    )

                                                                            # 데이터셋이 작은 경우 더 큰 값을 고려하십시오.
    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection, overwrite=True)  # 'max_id' 부분 삭제






[Oct 10, 17:26:21] #> Creating directory /content/experiments/notebook/indexes/test 


#> Starting...
#> Joined...


In [ ]:
# collection[0]
# 태양과 달이 대략 같은 크기로 보이는 이유는 달이 태양보다 지름이 더 짧고 지구에서 더 가까이 위치하기 때문입니다. 달은 지구와의 거리가 상대적으로 가까워서 우리 눈에는 태양과 달이 비슷한 크기로 보입니다. 이는 우연이 아니라 천문학적인 현상으로, 이러한 현상을 '달의 지름 효과'라고 합니다. 태양과 달이 대략 같은 크기로 보이는 이러한 현상은 우리에게 아름다운 일식과 월식을 선사해주는데, 이는 천문학적인 현상의 아름다움을 더욱 감상할 수 있는 기회를 제공합니다.

"태양과 달이 대략 같은 크기로 보이는 이유는 달이 태양보다 지름이 더 짧고 지구에서 더 가까이 위치하기 때문입니다. 달은 지구와의 거리가 상대적으로 가까워서 우리 눈에는 태양과 달이 비슷한 크기로 보입니다. 이는 우연이 아니라 천문학적인 현상으로, 이러한 현상을 '달의 지름 효과'라고 합니다. 태양과 달이 대략 같은 크기로 보이는 이러한 현상은 우리에게 아름다운 일식과 월식을 선사해주는데, 이는 천문학적인 현상의 아름다움을 더욱 감상할 수 있는 기회를 제공합니다."

In [ ]:
# docid_list[0]
# 3eff8a03-46a4-4e9e-8473-1e7f08b81a33

'3eff8a03-46a4-4e9e-8473-1e7f08b81a33'

In [ ]:
# src_list[0]
# ko_ai2_arc__ARC_Challenge__train

'ko_ai2_arc__ARC_Challenge__train'

In [ ]:
# 검색을 수행하기 위한 Searcher 객체를 생성합니다.
# ColBERT 인덱스를 기반으로 검색 작업을 수행합니다.

with Run().context(RunConfig(experiment='notebook')):  # 실행 환경을 설정합니다. 여기서는 'notebook'이라는 이름으로 실험을 정의합니다.
    searcher = Searcher(index=index_name, collection=collection)  # 지정된 인덱스와 컬렉션을 사용하여 Searcher 객체를 초기화합니다.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda

[Oct 10, 17:32:21] #> Loading codec...
[Oct 10, 17:32:21] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/usr/local/lib/python3.10/dist-packages/colbert/indexing/codecs/residual.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  centroids = torch.load(centroids_path, map_loc

[Oct 10, 17:32:21] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/colbert/search/index_loader.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ivf, ivf_lengths = torch.load(os.path.join(self.inde

[Oct 10, 17:32:21] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 301.14it/s]

[Oct 10, 17:32:21] #> Loading codes and residuals...



  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/colbert/indexing/codecs/residual_embeddings.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


### Colbert 색인 문서 검색

In [ ]:
# 문서 검색 test
queries = ["일식이 뭐야?"]  # 검색할 질의 목록입니다.

# 각 질의에 대해 검색을 수행합니다.
for query in queries:
    print(f"#> {query}")  # 현재 검색 질의를 출력합니다.

    # 검색 질의에 대한 상위 5개의 문서를 검색합니다.
    results = searcher.search(query, k=3)  # k는 검색 결과에서 반환할 상위 문서의 개수를 의미합니다.

    # 검색된 상위 k개의 문서를 출력합니다.
    for passage_id, passage_rank, passage_score in zip(*results):
        # 검색 결과의 순위, 점수 및 해당 문서를 출력합니다.
        print(f"{docid_list[passage_id]}\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")


#> 일식이 뭐야?

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . 일식이 뭐야?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  1463, 30019, 30022, 29997, 30019, 30020, 29999, 30019,
          100,  1029,   102,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


d7ea6ed2-014d-4710-b52c-114785584309	 [1] 		 24.0 		 혜성 꼬리는 태양에서 멀어지는 방향을 가리킵니다. 이는 혜성이 태양으로부터 멀어질 때, 태양의 열과 태양풍에 의해 꼬리가 형성되기 때문입니다. 혜성은 태양과의 궤도를 돌면서 태양으로부터 멀어지는 방향으로 이동합니다. 이러한 혜성의 꼬리는 태양으로부터 멀어질수록 길어지고 밝아집니다. 이는 태양의 열과 태양풍이 혜성의 표면을 가열하고 흩뿌리는 물질들이 꼬리로 형성되기 때문입니다. 따라서 혜성 꼬리는 태양에서 멀어지는 방향을 가리킵니다.
699c03db-6230-48fc-83b3-cece0812d294	 [2] 		 24.0 		 스트로브잣나무는 일반적으로 성장한 경우에는 높이가 100피트입니다. 그러나 미시간주 온토나곤에는 150피트에 이르는 스트로브잣나무가 있습니다. 따라서, 미시간의 스트로브잣나무는 일반적인 스트로브잣나무보다 1.5배 더 큽니다. 이는 미시간의 스트로브잣나무가 일반적인 스트로브잣나무의 높이에 비해 1.5배 더 높다는 것을 의미합니다.
56cdf9df-2ee0-475a-a31a-e27c8baf9ba8	 [3] 		 23.9 		 퇴적암은 풍화에 의해 형성되는 암석의 일종입니다. 퇴적암은 주로 강, 호수, 바다 등에서 퇴적물이 쌓여서 형성됩니다. 이러한 퇴적물은 풍화된 암석이나 토양에서 운반되어 수렴지에 쌓이게 됩니다. 그리고 시간이 지나면서 압력과 결합력에 의해 퇴적물은 고체로 변하고, 암석으로 변환됩니다. 따라서 퇴적암은 풍화에 의해 형성되는 암석의 대표적인 예입니다.


In [ ]:
# results
# (passage_id:[5, 3, 2, 4], passage_rank:[1, 2, 3, 4], passage_score:[26.296875, 24.96875, 24.96875, 24.453125])

([5, 3, 2, 4], [1, 2, 3, 4], [26.296875, 24.96875, 24.96875, 24.453125])

## RAG

### 필요 라이브러리 임포트 및 openai api 준비하기

In [ ]:
# 아래부터는 실제 RAG를 구현하는 코드입니다.
from openai import OpenAI  # OpenAI API를 사용하기 위한 모듈 가져오기
import traceback  # 예외 발생 시 오류 추적을 위한 모듈 가져오기
from dotenv import load_dotenv
import os

# OPENAI_API_KEY.env 파일 생성
# OPENAI_API_KEY = "your API key here"

# 특정 .env 파일 경로를 지정하여 환경 변수 로드
load_dotenv(dotenv_path='/content/drive/MyDrive/colbert_ir/OPENAI_API_KEY.env')

# OpenAI API 키 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")

# 이후 OpenAI API 키를 사용할 코드 작성
# print(openai_api_key)  # API 키를 확인하기 위한 출력 (실제 코드에서는 제거)
client = OpenAI()  # OpenAI 클라이언트 생성
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-3.5-turbo-1106"

In [ ]:
# RAG 구현에 필요한 Question Answering을 위한 LLM 프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",  # 함수 타입 설정
        "function": {
            "name": "search",  # 함수 이름 설정
            "description": "search relevant documents",  # 함수 설명 설정
            "parameters": {
                "properties": {  # 파라미터의 속성 설정
                    "standalone_query": {
                        "type": "string",  # 파라미터의 데이터 타입을 'string'으로 설정
                        "description": "Final query suitable for use in search from the user messages history."  # 사용자 메시지 히스토리로부터 검색에 사용할 최종 질의 설명
                        # "description": "사용자 메세지 기록으로부터 검색에 사용할 적합한 최종 질의"
                    }
                },
                "required": ["standalone_query"],  # 필수 파라미터로 'standalone_query' 설정
                "type": "object"  # 파라미터 전체 타입을 'object'로 설정
            }
        }
    },
]


In [ ]:
# 대화형 에이전트를 위한 대화 관리자(dialog manager) 구현

# 대화형 에이전트가 사용자와 상호작용하면서 검색 기능을 수행하는 함수입니다.
def conversation_search(messages, persona, tools):
    # 시스템 메시지와 사용자 메시지를 결합하여 메시지 리스트를 생성합니다.
    msg = [{"role": "system", "content": persona}] + messages

    # OpenAI의 Chat API를 사용하여 대화 응답을 생성합니다.
    result = client.chat.completions.create(
        model=llm_model,  # 사용할 모델을 지정합니다.
        messages=msg,  # 구성된 메시지 리스트를 전달합니다.
        tools=tools,  # 사용할 도구 목록을 전달합니다.
        temperature=0,  # 답변의 일관성을 높이기 위해 온도를 0으로 설정합니다.
        seed=1  # 결과의 재현성을 유지하기 위해 랜덤 시드를 설정합니다.
    )

    # 생성된 응답 메시지를 변수에 저장합니다.
    response_message = result.choices[0].message

    # 만약 응답 메시지에서 도구 호출이 포함된 경우
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            # 도구 호출에 전달된 인자를 JSON 형식으로 파싱합니다.
            function_args = json.loads(tool_call.function.arguments)
            standalone_query = function_args.get("standalone_query")  # 독립적인 검색 쿼리를 추출합니다.

            # 검색 쿼리를 이용해 상위 3개의 결과를 검색합니다.
            results = searcher.search(standalone_query, k=10)
            print("검색어", standalone_query)
            print()
            print("검색 결과 :")
            retrieved_context = []
            for passage_id, passage_rank, passage_score in zip(*results):
                retrieved_context.append(searcher.collection[passage_id])
                print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

            msg = [{"role": "system", "content": persona_qa}] + messages
            msg.append({"role": "user", "content": json.dumps(retrieved_context)})
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=1
                )
            print("LLM 대답>>:")
            print(qaresult.choices[0].message.content)
    else:
        print("LLM 대답>>:")
        print(response_message.content)

In [ ]:
# 사용자와의 대화 메시지 목록을 정의합니다.
messages = [{"role": "user", "content": "광합성이 뭐니"},
            {"role": "user", "content": "메탄과 산소의 화학 반응"}
    # {"role": "user", "content": "안녕하세요, 저는 새로운 개 주인이고 방금 영국 마스티프를 입양했어요. 그를 돌보는 데 대해 몇 가지 팁을 줄 수 있나요?"},
    # # 사용자가 새로운 개 주인임을 밝히고, 개를 돌보는 데 대한 조언을 요청했습니다.
    # {"role": "assistant", "content": "물론이죠! 영국 마스티프 같은 대형견을 돌보는 것은 매우 보람찬 일입니다. 그들은 균형 잡힌 식단, 규칙적인 운동, 그리고 정기적인 수의사 검진이 필요합니다. 그들의 돌봄에 대해 구체적으로 궁금한 점이 있나요?"},
    # # 어시스턴트가 영국 마스티프 돌봄에 대한 일반적인 정보를 제공하며 추가적인 질문을 유도합니다.
    # {"role": "user", "content": "네, 특히 그의 식단에 대해 걱정이 되네요. 1살 영국 마스티프에게 얼마나 많은 양을 먹여야 하나요?"},
    # # 사용자가 구체적으로 영국 마스티프의 식단에 대한 질문을 하고 있습니다.
]

# 대화형 에이전트를 사용하여 반려 동물과 관련된 질의에 응답하도록 합니다.
conversation_search(messages, persona_function_calling, tools)


검색어 메탄과 산소의 화학 반응

검색 결과 :
	 [1] 		 27.3 		 말-마차 시스템이 휴식 상태에서 가속하는 방법은 지면의 전방 정지마찰력이 마차에 작용하는 후방 마찰력보다 크기 때문입니다. 이러한 조건에서 전방 가속이 제공됩니다. 정지 상태의 마차에 연결된 말은 마차와 함께 움직이기 위해 힘을 가해야 합니다. 이때, 지면의 전방 정지마찰력이 후방 마찰력보다 크면, 말은 마차를 앞으로 밀어내는 힘을 발휘하게 됩니다. 이러한 힘의 조합으로 말-마차 시스템은 휴식 상태에서 가속할 수 있습니다. 따라서, 말-마차 시스템이 가속하기 위해서는 전방 정지마찰력이 후방 마찰력보다 크도록 조절되어야 합니다.
	 [2] 		 27.3 		 미토콘드리아는 세포 내에서 중요한 역할을 수행합니다. 이 작고 원형의 세포 소기관은 에너지 생산에 관여합니다. 미토콘드리아는 설탕을 분해하여 에너지를 방출합니다. 이 과정은 호흡 과정이라고도 알려져 있습니다. 세포 내에서 에너지는 생존과 기능 수행에 필수적입니다. 따라서 미토콘드리아는 세포의 생존과 기능 수행에 중요한 역할을 담당하고 있습니다.
	 [3] 		 27.2 		 말과 코뿔소는 공통 조상을 공유합니다. 이는 코뿔소와 말이 매우 유사한 소화계와 발을 가지고 있기 때문입니다. 둘 다 홀수 개의 발가락을 가지고 있지만, 말은 하나의 발가락을 가지고 있고, 코뿔소는 세 개의 발가락을 가지고 있습니다. 이러한 유사한 특징들은 코뿔소와 말이 공통 조상으로부터 유래했음을 시사합니다. 코뿔소와 말은 서로 다른 환경에서 진화하였지만, 공통 조상으로부터 이어져온 유사한 특징들을 가지고 있습니다. 따라서, 코뿔소와 말은 관련이 있으며, 공통 조상을 공유한다고 할 수 있습니다.
	 [4] 		 27.1 		 모래 입자는 물보다 밀도가 높아서 용기 바닥에 가라앉았습니다. 이는 모래와 물의 입자 크기와 형태의 차이로 인해 발생합니다. 모래 입자는 물 입자보다 더 무거우며, 물보다 더 밀집되어 있습니다. 따라서, 모래를 물에 넣고 저으면 모래 입

In [ ]:
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}  # 초기 응답 딕셔너리 생성 (질의, 상위 결과, 참조, 답변 필드 포함)

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages  # 시스템 메시지와 사용자 메시지를 결합하여 msg 생성
    try:
        result = client.chat.completions.create(  # OpenAI 클라이언트의 chat completion 생성 호출
            model=llm_model,  # 사용할 LLM 모델 설정
            messages=msg,  # 대화 메시지 전달
            tools=tools,  # 사용할 함수 리스트 전달
            # tool_choice={"type": "function", "function": {"name": "search"}},  # 사용 가능한 함수 중 하나를 선택하는 옵션 (주석 처리됨)
            temperature=0,  # 응답의 무작위성 설정 (0이면 더 결정적인 응답 생성)
            seed=1,  # 무작위성 제어를 위한 시드 설정
            timeout=10  # 요청 타임아웃 시간 설정 (초 단위)
        )
    except Exception as e:
        traceback.print_exc()  # 예외 발생 시 오류 추적 출력
        return response  # 오류 발생 시 초기화된 빈 응답 반환

    # # 검색이 필요한 경우 검색 호출 후 결과를 활용하여 답변 생성
    # if result.choices[0].message.tool_calls:  # LLM이 함수 호출을 요청한 경우
    #     tool_call = result.choices[0].message.tool_calls[0]  # 첫 번째 함수 호출 정보 가져오기
    #     function_args = json.loads(tool_call.function.arguments)  # 함수 인자를 JSON 형식으로 파싱
    #     standalone_query = function_args.get("standalone_query")  # standalone_query 추출

    # 생성된 응답 메시지를 변수에 저장합니다.
    response_message = result.choices[0].message

    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            # 도구 호출에 전달된 인자를 JSON 형식으로 파싱합니다.
            function_args = json.loads(tool_call.function.arguments)
            standalone_query = function_args.get("standalone_query")  # 독립적인 검색 쿼리를 추출합니다.


            # Baseline으로는 sparse_retrieve만 사용하여 검색 결과 추출
            results = searcher.search(standalone_query, k=3)  # sparse_retrieve 함수를 호출하여 검색 결과 가져오기

            response["standalone_query"] = standalone_query  # standalone_query 응답에 추가
            retrieved_context = []

            for passage_id, passage_rank, passage_score in zip(*results):  # 검색된 문서들을 순회
                retrieved_context.append(searcher.collection[passage_id])  # 검색된 문서의 content 필드 추가
                response["topk"].append(docid_list[passage_id])  # 검색된 문서의 docid 추가
                response["references"].append({"score": passage_score, "content": searcher.collection[passage_id]})  # 검색된 문서의 점수 및 내용 추가
                print(passage_id, passage_rank, passage_score)
                                    #print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

                msg = [{"role": "system", "content": persona_qa}] + messages  # 질문 응답 프롬프트와 함께 메시지 결합
                msg.append({"role": "user", "content": json.dumps(retrieved_context)})  # assistant 역할로 검색된 컨텍스트 추가

        try:
            qaresult = client.chat.completions.create(  # OpenAI 클라이언트를 사용해 최종 답변 생성
                model=llm_model,  # 사용할 언어 모델 설정 (여기서는 gpt-3.5-turbo-1106)
                messages=msg,  # 대화에 사용할 메시지 리스트 전달
                temperature=0,  # 생성된 응답의 무작위성 설정 (0은 더 결정적인 응답 생성)
                seed=1,  # 무작위성 제어를 위한 시드 설정
                timeout=30  # 요청에 대한 타임아웃 설정 (30초)
            )
        except Exception as e:
            traceback.print_exc()  # 예외 발생 시 오류 추적 출력
            return response  # 오류 발생 시 초기화된 응답 반환

        response["answer"] = qaresult.choices[0].message.content  # 최종 생성된 답변 응답에 추가

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content  # 검색 없이 생성된 답변 추가

    return response  # 최종 답변 반환


In [ ]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출 후 파일에 저장
def eval_rag(eval_filename, output_filename):  # 평가 파일과 출력 파일을 입력으로 받음
    with open(eval_filename) as f, open(output_filename, "w") as of:  # 평가 파일 읽기 모드, 출력 파일 쓰기 모드로 열기
        idx = 0  # 평가 데이터의 인덱스를 0으로 초기화
        for line in f:  # 평가 파일의 각 줄을 순회
            # if idx > 5:
            #     break  # 특정 개수 이상 평가를 제한할 때 사용 (현재 주석 처리됨)
            j = json.loads(line)  # 평가 데이터의 한 줄을 JSON 형식으로 로드
            print(f'Test {idx}\nQuestion: {j["msg"]}')  # 현재 인덱스와 질문 출력
            response = answer_question(j["msg"])  # 질문에 대한 답변 생성
            print(f'Answer: {response["answer"]}\n')  # 생성된 답변 출력

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}  # 평가 결과 저장할 딕셔너리 생성
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')  # 평가 결과를 JSON 형식으로 출력 파일에 저장
            idx += 1  # 인덱스를 1 증가

In [ ]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("/content/drive/MyDrive/colbert_ir/eval.jsonl", "/content/drive/MyDrive/colbert_ir/ColBERT_submission.csv")  # 평가 파일 경로와 출력 파일명을 입력으로 하여 eval_rag 함수 호출

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
1943 1 27.140625
1272 2 27.0625
810 3 27.0625
Answer: 주어진 정보에 따르면, 나무의 분류를 조사하기 위한 방법은 다양합니다. 먼저 나무의 잎 모양, 가지 구조, 꽃과 열매의 존재 여부 등을 관찰하여 나무를 식별할 수 있습니다. 또한, 나무의 표고나 통나무의 단면을 관찰하여 식별할 수도 있습니다. 또한, 나무의 특징을 토대로 식별표를 활용하여 나무를 식별할 수도 있습니다. 이러한 방법들을 활용하여 나무의 분류를 조사할 수 있습니다.

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
1892 1 28.171875
2089 2 28.125
3473 3 28.09375
Answer: 죄송합니다, 주어진 정보로는 각 나라에서의 공교육 지출 현황에 대한 구체적인 정보를 제공할 수 없습니다. 더 많은 정보가 필요하거나 다른 출처의 데이터가 필요할 것으로 보입니다. 원하시는 정보를 얻기 위해서는 해당 국가의 교육부나 통계청 등의 공식 기관에서 발표한 자료를 참고하는 것이 도움이 될 것입니다.

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
186 1 27.5625
3165 2 27.28125
423 3 27.234375
Answer: 죄송합니다, 제가 이 정보를 활용하여 적절한 답변을 제공할 수 없습니다. 더 많은 정보가 필요하거나 다른 주제에 대해 질문해 주시겠어요?

Test 3
Question: [{'role': 'user', 'content': '통학 버